In [1]:
import pandas as pd
import numpy as np
import os
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

# Reading Soure data

In [2]:
source = 'source/Data.csv'

In [3]:
# before reading into python, you can open the source file with desktop spreadsheet programmes like Excel
# see if there are empty rows and columns we could drop and provide the correct options to pandas.

# sometimes the file is not encoded in UTF8, and pandas will fail to read it. 
# In this case can try 'iso-8859-1' encoding.

data = pd.read_csv(source, encoding='iso-8859-1', skiprows=1)

# explore source data

usually the data will be in one of these two forms:

1. country/time/other entity concepts as columns, and each measure type takes one column too, just like what we will deal with in this dataset. An example of this form of data:

    ```
    | country | age group | year | measure 1 | measure 2 | ... |
    | afg     |       15+ | 1990 |       1.2 |       2.3 | ... |
    | ...     |           |      |           |           |     |
    | zwe     |       65+ | 2016 |        20 |     15.23 | ... |
```
2. country/measure type/entity type/ as columns, and each year takes one column. The format will be

    ```
    | country | age group | indicator  | 1990 | ... |  2016 |
    | afg     |       15+ | measure 1  |  1.2 | ... |    20 |
    | afg     |       15+ | measure 2  |  2.3 | ... |    99 |
    | ...     |           |            |      |     |       |
    | zwe     |       65+ | measure 20 |   20 | ... | 15.23 |

    ```
    
So we need to see what is the source format and decide what is the best way to extract the data. Sometimes there will be a codebook provided together with source data, in which we can find more details of the data. But in this dataset we just look at the data and find out what we need.

In [6]:
pd.set_option('max_rows', 15)

data

,year,disaster type,iso,country_name,occurrence,Total deaths,Affected,Injured,Homeless,Total affected,Total damage
0,1900,Drought,CPV,Cabo Verde,1,11000.0,NaN,NaN,NaN,NaN,NaN
1,1900,Drought,IND,India,1,1250000.0,NaN,NaN,NaN,NaN,NaN
2,1900,Epidemic,JAM,Jamaica,1,30.0,NaN,NaN,NaN,NaN,NaN
3,1900,Flood,JAM,Jamaica,1,300.0,NaN,NaN,NaN,NaN,NaN
4,1900,Storm,USA,United States of America (the),1,6000.0,NaN,NaN,NaN,NaN,30000.0
5,1900,Volcanic activity,JPN,Japan,1,30.0,NaN,NaN,NaN,NaN,NaN
6,1901,Earthquake,JPN,Japan,1,18.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
12509,2015,Wildfire,AUS,Australia,3,2.0,NaN,164.0,453.0,617.0,351000.0
12510,2015,Wildfire,CAN,Canada,1,NaN,13000.0,NaN,36.0,13036.0,NaN


In [7]:
data.columns

Index(['year', 'disaster type', 'iso', ' country_name', 'occurrence',
       'Total deaths', 'Affected', 'Injured', 'Homeless', 'Total affected',
       'Total damage'],
      dtype='object')

We can see in above table that:

1. there are 2 entity domains: disaster type and country
2. measure type concepts are 'occurrence', 'Total deaths', 'Affected', 'Injured', 'Homeless', 'Total affected', 'Total damage'. Their units are all numbers of people, so we should assume they are integer, not floating point numbers. Here they displayed as folating point, because there are n/a values, which are treated as floating point in pandas.
3. N/A values are in measures, which we should drop them later

In [9]:
data.columns = list(map(str.strip, data.columns))  # remove spaces in beginning and end of column names

# extract entities

In [10]:
# country

country = data[['iso', 'country_name']].copy()  # we should call .copy() to avoid overwriting the source data

In [11]:
country.columns = ['country', 'name']  # rename column names to match DDF model

In [12]:
country['country'] = country['country'].map(to_concept_id)  # to_concept_id() makes a string to alphanumberic form

In [14]:
country = country.drop_duplicates()

In [15]:
country

,country,name
0,cpv,Cabo Verde
1,ind,India
2,jam,Jamaica
4,usa,United States of America (the)
5,jpn,Japan
7,tza,"Tanzania, United Republic of"
8,uga,Uganda
...,...,...
7659,shn,"Saint Helena, Ascension and Tristan da Cunha"
8037,mnp,Northern Mariana Islands (the)


In [16]:
country.to_csv('../ddf--entities--country.csv', index=False)  # note: the index is not needed so index=False

In [ ]:
# disaster type, basically same process as country

In [17]:
disas = data[['disaster type']].copy()

In [18]:
disas['disaster'] = disas['disaster type'].map(to_concept_id)

In [19]:
disas.columns = ['name', 'disaster']

In [20]:
disas.drop_duplicates()[['disaster', 'name']].to_csv('../ddf--entities--disaster.csv', index=False)

# extract concepts

In [ ]:
# concept

In [21]:
concs = data.columns[4:]   # all measure types

In [22]:
concs = [*['Year', 'Name', 'disaster', 'country'], *concs]  # manually add discrete types

In [23]:
concs

['Year',
 'Name',
 'disaster',
 'country',
 'occurrence',
 'Total deaths',
 'Affected',
 'Injured',
 'Homeless',
 'Total affected',
 'Total damage']

In [24]:
# construct the concepts DataFrame

In [25]:
con = pd.DataFrame([], columns=['concept', 'name', 'concept_type'])

In [26]:
con['name'] = concs

In [27]:
con['concept'] = con['name'].map(to_concept_id)

In [28]:
# setting the concept_type of concepts.

con['concept_type'] = 'measure'

con['concept_type'].iloc[0] = 'time'
con['concept_type'].iloc[1] = 'string'
con['concept_type'].iloc[2] = 'entity_domain'
con['concept_type'].iloc[3] = 'entity_domain'

In [29]:
con.to_csv('../ddf--concepts.csv', index=False)

# Datapoints

target data point format for this dataset should be:

```
| country | disaster_type | year | measure |
| afg     | flood         | 1990 |       1 |
| ...     |               |      |         |
| zwe     | wildfire      | 2015 |       2 |
```

because each measure takes one column, we can set country/disaster_type/year as index and iterate each other columns to get datapoints for DDF.

In [30]:
dps = data.drop('country_name', axis=1)

In [31]:
dps.columns = list(map(to_concept_id, dps.columns))

In [32]:
dps = dps.rename(columns={'disaster_type': 'disaster', 'iso': 'country'})

In [33]:
dps['country'] = dps['country'].map(to_concept_id)
dps['disaster'] = dps['disaster'].map(to_concept_id)

In [34]:
# set the index to country/disaster/year

dps = dps.sort_values(by=['country', 'disaster', 'year']).set_index(['country', 'disaster', 'year'])

In [35]:
for name, col in dps.items():  # iterate each column with index.
    
    df = col.reset_index()
    path = os.path.join('../', 'ddf--datapoints--{}--by--country--disaster--year.csv'.format(name))
    df = df.dropna()
    df[name] = df[name].map(int)  # 
    
    df.to_csv(path, index=False)

In [ ]:
# And at last, create an index file

In [36]:
create_index_file('../')

,key,value,file
0,concept,name,ddf--concepts.csv
1,concept,concept_type,ddf--concepts.csv
0,"country,disaster,year",affected,ddf--datapoints--affected--by--country--disast...
0,"country,disaster,year",homeless,ddf--datapoints--homeless--by--country--disast...
0,"country,disaster,year",injured,ddf--datapoints--injured--by--country--disaste...
0,"country,disaster,year",occurrence,ddf--datapoints--occurrence--by--country--disa...
0,"country,disaster,year",total_affected,ddf--datapoints--total_affected--by--country--...
0,"country,disaster,year",total_damage,ddf--datapoints--total_damage--by--country--di...
0,"country,disaster,year",total_deaths,ddf--datapoints--total_deaths--by--country--di...
0,country,name,ddf--entities--country.csv


# Validation

after we finish creating all DDF files, we can run the validation tool to check our dataset

In [44]:
!validate-ddf --exclude-rules "NON_DDF_FOLDER" ../   # no output means they passed all tests by validation tool.

[
{}]

